# Semantic Model

This notebook focuses on building a semantic model for HR data analysis. It includes functions to process and enrich data, calculate key metrics such as absence rates and employee counts, and generate insights like contract type distribution and gender distribution. The data is loaded from a SQLite database and analyzed using pandas.

In [ ]:
import datetime
import sqlite3

import pandas as pd

# Connexion à la base SQLite
conn = sqlite3.connect("fabric_sim.db")


def load_data(conn):
    """
    Load and preprocess data from the SQLite database.

    Parameters:
    conn (sqlite3.Connection): SQLite database connection object.

    Returns:
    DataFrame: A merged and preprocessed DataFrame containing contract and absence data,
               with calculated total absence days and absence rates.
    """
    absence = pd.read_sql("SELECT * FROM absence", conn)
    contract = pd.read_sql("SELECT * FROM contract_basis", conn)

    df = contract.merge(
        absence, on=["firm_id", "person_id", "department_id", "category_id"], how="left"
    )

    # Identify absence columns by code
    absence_columns = [
        col for col in df.columns if col.startswith("qty_") and col.endswith("_days")
    ]

    # Clean and convert absence columns
    df[absence_columns] = df[absence_columns].fillna(0)
    for col in absence_columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Calculate total absence days for all causes
    df["total_absence_days"] = df[absence_columns].sum(axis=1)

    # Calculate absence rate
    df["absence_rate"] = df["total_absence_days"] / df["qty_working_days"]
    df["absence_rate"] = df["absence_rate"].replace([float("inf"), -float("inf")], None)

    return df


def load_absence_type(conn):
    """
    Load the absence_type table (absence code reference data).

    Parameters:
    conn (sqlite3.Connection): SQLite database connection object.

    Returns:
    DataFrame: A DataFrame containing absence type reference data.
    """
    return pd.read_sql("SELECT * FROM absence_type", conn)


df = load_data(conn)

df.head(10)

,contract_zip_code,firm_id,department_id,category_id,person_id,contract_start_date,contract_end_date,company_start_date,birth_date,contract_terminatio_reason,...,freq_p0_days,freq_p1_days,freq_p2_days,freq_p3_days,freq_a1_days,freq_a2_days,qty_days_worked,qty_working_days,total_absence_days,absence_rate
0,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000
1,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,30.0,1.428571
2,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,23.0,23.0,1.000000
3,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000
4,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,22.0,22.0,1.000000
5,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,22.0,22.0,1.000000
6,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000
7,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,23.0,23.0,1.000000
8,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,22.0,22.0,1.000000
9,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000


In [ ]:
# === MESURES SÉMANTIQUES ===


def total_employees_by_firm(df):
    """
    Calculate the total number of employees for each firm.

    Parameters:
    df (DataFrame): The input DataFrame containing employee data, including 'firm_id' and 'person_id' columns.

    Returns:
    DataFrame: A DataFrame with 'firm_id' and the corresponding count of unique employees ('employee_count').
    """
    return (
        df.groupby("firm_id")["person_id"].nunique().reset_index(name="employee_count")
    )


def average_absence_rate_by_firm(df):
    """
    Calculate the average absence rate for each firm.

    Parameters:
    df (DataFrame): The input DataFrame containing employee data, including 'firm_id' and 'absence_rate' columns.

    Returns:
    DataFrame: A DataFrame with 'firm_id' and the corresponding average absence rate.
    """
    return df.groupby("firm_id")["absence_rate"].mean().reset_index()


def contract_type_distribution_by_firm(df):
    """
    Calculate the distribution of contract types for each firm.

    Parameters:
    df (DataFrame): The input DataFrame containing employee data, including 'firm_id', 'contract_type', and 'person_id' columns.

    Returns:
    DataFrame: A DataFrame with 'firm_id', 'contract_type', and the count of employees for each contract type.
    """
    return (
        df.groupby(["firm_id", "contract_type"])["person_id"]
        .count()
        .reset_index(name="count")
    )


def filter_by_contract_type(df, contract_type="CDI"):
    """
    Filter the DataFrame to include only employees with a specific contract type.

    Parameters:
    df (DataFrame): The input DataFrame containing employee data, including a 'contract_type' column.
    contract_type (str): The contract type to filter by (default is "CDI").

    Returns:
    DataFrame: A filtered DataFrame containing only rows with the specified contract type.
    """
    return df[df["contract_type"] == contract_type]


# === ENRICHISSEMENT AVEC absence_type ===


def enrich_absence_with_type(absence_df, absence_type_df):
    """
    Transforms the qty_*_days columns into a long format and links them to absence types.

    Parameters:
    absence_df (DataFrame): The input DataFrame containing absence data, including columns like 'qty_*_days'.
    absence_type_df (DataFrame): The input DataFrame containing absence type reference data,
                                 including a 'type_absence' column.

    Returns:
    DataFrame: A DataFrame in long format with absence days linked to their respective absence types,
               including columns like 'type_absence_code', 'days', and 'type_absence_fr'.
    """
    qty_columns = [
        col
        for col in absence_df.columns
        if col.startswith("qty_") and col.endswith("_days")
    ]

    melted = absence_df.melt(
        id_vars=[
            "firm_id",
            "department_id",
            "category_id",
            "person_id",
            "year",
            "month",
        ],
        value_vars=qty_columns,
        var_name="absence_code_col",
        value_name="days",
    )

    # Extract the absence code (e.g., qty_p1_days → P1)
    melted["type_absence_code"] = (
        melted["absence_code_col"]
        .str.extract(r"qty_([a-z0-9]+)_days", expand=False)
        .str.upper()
    )

    # Rename column for merging
    absence_type_df = absence_type_df.rename(
        columns={"type_absence": "type_absence_code"}
    )

    # Merge with reference data
    enriched = melted.merge(absence_type_df, on="type_absence_code", how="left")

    # Keep only actual absences
    enriched = enriched[enriched["days"] > 0]

    return enriched


def absences_by_type(enriched_absences_df):
    """
    Aggregate absences by type for display.

    Parameters:
    enriched_absences_df (DataFrame): The input DataFrame containing enriched absence data,
                                      including 'type_absence_fr' and 'days' columns.

    Returns:
    DataFrame: A DataFrame with 'type_absence_fr' and the total 'days' of absences for each type,
               sorted in descending order by 'days'.
    """
    return (
        enriched_absences_df.groupby("type_absence_fr")["days"]
        .sum()
        .reset_index()
        .sort_values(by="days", ascending=False)
    )


def gender_distribution_by_firm(df):
    """
    Calculate the gender distribution of employees for each firm.

    Parameters:
    df (DataFrame): The input DataFrame containing employee data, including 'firm_id' and 'gender' columns.

    Returns:
    DataFrame: A DataFrame with 'firm_id', 'gender', and the corresponding count of employees for each gender.
    """
    return (
        df.groupby(["firm_id", "gender"])["person_id"]
        .nunique()
        .reset_index(name="employee_count")
    )


def average_age_by_firm(df):
    """
    Calculate the average age of employees for each firm.

    Parameters:
    df (DataFrame): The input DataFrame containing employee data, including a 'birth_date' column.

    Returns:
    DataFrame: A DataFrame with 'firm_id' and the corresponding 'average_age' of employees.
    """
    today = pd.Timestamp(datetime.today())
    df["birth_date"] = pd.to_datetime(df["birth_date"], errors="coerce")
    df["age"] = ((today - df["birth_date"]).dt.days / 365.25).round(1)
    return df.groupby("firm_id")["age"].mean().reset_index(name="average_age")